In [1]:
# Calculate Feature Importance

# Import necessary dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model_evaluation_utils as meu
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

# Load and merge datasets # white = control; red = stroke; wine = data
stroke_data = pd.read_csv('Stroke/Injured Participants Data.csv', delim_whitespace=False)
control_data = pd.read_csv('Healthy Control Participants Data.csv', delim_whitespace=False)

# store wine type as an attribute
stroke_data['data_type'] = 'stroke'   
control_data['data_type'] = 'control'

# merge control and stroke data
datas = pd.concat([stroke_data, control_data])
#datas = datas.sample(frac=1, random_state=42).reset_index(drop=True)

# Prepare Training and Testing Datasets
features = datas.iloc[:,:-1]
feature_names = features.columns
class_labels = np.array(datas['data_type'])

X_data = datas.iloc[:,:-1]
y_label = datas.iloc[:,-1]

# Data Normalization
ss = StandardScaler().fit(X_data)
X = ss.transform(X_data)
le = LabelEncoder()
le.fit(y_label)
y = le.transform(y_label)

In [2]:
feature_names

Index(['Abs error X', 'Abs error Y', 'Abs error XY', 'Variability X',
       'Variability Y', 'Variability XY', 'Contraction expansion ratio X',
       'Contraction expansion ratio Y', 'Contraction expansion ratio XY',
       'Shift X', 'Shift Y', 'Shift XY'],
      dtype='object')

In [3]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

kfold = KFold(n_splits=10, random_state=42, shuffle=True)
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

In [6]:
# Model Interpretation
# View Feature importances
from sklearn.linear_model import LogisticRegression
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel

kfold = KFold(n_splits=10, random_state=42, shuffle=True)

lr_feature_importance = []
for i, (train_lr,test_lr) in enumerate(kfold.split(X,y)):
    plt.figure()
    model_lr = LogisticRegression().fit(X[train_lr],y[train_lr])
    lr_interpretation = Interpretation(X[test_lr], feature_names=feature_names)
    lr_in_model = InMemoryModel(model_lr.predict_proba, examples=X[train_lr], target_names=model_lr.classes_)
    lr_fea = lr_interpretation.feature_importance.feature_importance(lr_in_model, ascending=False)
    #print(lr_fea)
    lr_feature_importance.append(lr_fea)
    #plots = lr_interpretation.feature_importance.plot_feature_importance(lr_in_model, ascending=False)
    #plt.xlabel('Relative Importance Score')
    #plt.ylabel('Score')
print(lr_feature_importance)    


[12/12] features ████████████████████ Time elapsed: 0 seconds[Variability Y                     0.229826
Variability XY                    0.139193
Contraction expansion ratio XY    0.135825
Contraction expansion ratio X     0.102554
Variability X                     0.102126
Abs error XY                      0.088579
Contraction expansion ratio Y     0.083075
Abs error Y                       0.035162
Shift Y                           0.027156
Shift X                           0.020914
Abs error X                       0.020652
Shift XY                          0.014939
dtype: float64, Variability Y                     0.211947
Variability XY                    0.133165
Contraction expansion ratio XY    0.127138
Contraction expansion ratio Y     0.101361
Contraction expansion ratio X     0.098091
Abs error XY                      0.093328
Variability X                     0.091862
Shift Y                           0.050618
Abs error Y                       0.034848
Abs error X        

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [35]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold

#train model
#model_dt = DecisionTreeClassifier(max_depth=4)
KF_dt = KFold(n_splits=10, shuffle=True)

output_dt = cross_validate(estimator=model_dt,X=X,y=y,cv=kfold,scoring=scoring)
#model_dt.fit(X,y)
for idx,(train_dt,test_dt) in enumerate(KF_dt.split(X,y)):
    model_dt = DecisionTreeClassifier(max_depth=4).fit(X[train_dt], y[train_dt])
    print("Features sorted by their score for estimator {}:".format(idx))
    #feature_importances = model_dt.feature_importances_
    feature_importances = pd.DataFrame(model_dt.feature_importances_,
                                       index = feature_names,
                                        columns=['importance']).sort_values('importance', ascending=False)
    print(feature_importances)
    
mean_fea = feature_importances.mean()
std_fea = feature_importances.std()
print(mean_fea)
print(std_fea)

Features sorted by their score for estimator 0:
                                importance
Variability Y                     0.721365
Variability XY                    0.100459
Contraction expansion ratio X     0.079755
Variability X                     0.026343
Abs error Y                       0.020696
Shift Y                           0.016073
Contraction expansion ratio XY    0.012261
Contraction expansion ratio Y     0.009673
Shift X                           0.007448
Abs error XY                      0.005925
Abs error X                       0.000000
Shift XY                          0.000000
Features sorted by their score for estimator 1:
                                importance
Variability Y                     0.698791
Variability X                     0.098815
Contraction expansion ratio X     0.078418
Shift Y                           0.048928
Abs error Y                       0.041553
Shift X                           0.012956
Abs error X                       0.011903
C

In [41]:
# https://machinelearningmastery.com/how-to-configure-k-fold-cross-validation/
# Support Vector Machine
from sklearn.ensemble import RandomForestClassifier

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = KFold(n_splits=10, random_state=42, shuffle=True)
model_rf=RandomForestClassifier(n_estimators=50) 

output_rf = cross_validate(estimator=model_rf, X=X, y=y, cv=kfold, scoring=scoring, return_estimator=True)
for idx,estimator in enumerate(output_rf['estimator']):
    print("Features sorted by their score for estimator {}:".format(idx))
    feature_importances = pd.DataFrame(estimator.feature_importances_,
                                       index = feature_names,
                                        columns=['importance']).sort_values('importance', ascending=False)
    print(feature_importances)


Features sorted by their score for estimator 0:
                                importance
Variability Y                     0.189013
Variability XY                    0.157332
Variability X                     0.113826
Abs error Y                       0.092027
Abs error XY                      0.083536
Abs error X                       0.072603
Contraction expansion ratio XY    0.065596
Contraction expansion ratio X     0.057370
Shift Y                           0.044603
Contraction expansion ratio Y     0.044469
Shift X                           0.039991
Shift XY                          0.039632
Features sorted by their score for estimator 1:
                                importance
Variability Y                     0.195948
Variability XY                    0.147281
Variability X                     0.127914
Abs error XY                      0.107379
Contraction expansion ratio X     0.067593
Contraction expansion ratio Y     0.059024
Abs error X                       0.056645
C

In [32]:
# train the Random Forest Hyperparameter Tuning
from sklearn.ensemble import RandomForestClassifier

model_rft = RandomForestClassifier(n_estimators=200, max_features='auto')

results_rft = cross_validate(estimator=model_rft,
                          X=X,#X=features,
                          y=y,#y=labels,
                          cv=kfold,
                          scoring=scoring,
                            return_estimator=True)
for i, estimator_rft in enumerate(results_rft['estimator']):
    print("Feature Sorted by their score for estimator {}".format(i))
    feature_importance_rft = pd.DataFrame(estimator_rft.feature_importances_,
                                         index=feature_names,
                                         columns=['importance']).sort_values('importance', ascending=False)
    print(feature_importance_rft)


Feature Sorted by their score for estimator 0
                                importance
Variability Y                     0.202479
Variability XY                    0.157216
Variability X                     0.119167
Abs error XY                      0.085937
Contraction expansion ratio XY    0.073243
Abs error Y                       0.071682
Abs error X                       0.065456
Contraction expansion ratio X     0.059131
Contraction expansion ratio Y     0.048072
Shift Y                           0.041686
Shift XY                          0.039517
Shift X                           0.036414
Feature Sorted by their score for estimator 1
                                importance
Variability Y                     0.199328
Variability XY                    0.153088
Variability X                     0.124703
Abs error XY                      0.098609
Abs error Y                       0.077700
Contraction expansion ratio XY    0.063451
Contraction expansion ratio X     0.060425
Contr

In [45]:
from sklearn.svm import SVC
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel

kf_svm = KFold(n_splits=10, shuffle=True)
for i, (train_svm,test_svm) in enumerate(kf_svm.split(X,y)):
    model_svm = SVC(probability=True).fit(X[train_svm], y[train_svm])
    svm_interpretation = Interpretation(X[test_svm], feature_names=feature_names)
    svm_in_memory = InMemoryModel(model_svm.predict_proba, examples=X[train_svm], target_names=model_svm.classes_)
    svm_feature_importance = svm_interpretation.feature_importance.feature_importance(svm_in_memory, ascending=False)
    print(svm_feature_importance)
    

[12/12] features ████████████████████ Time elapsed: 0 secondsVariability Y                     0.220494
Variability XY                    0.159951
Variability X                     0.144340
Abs error Y                       0.095397
Abs error XY                      0.086147
Shift Y                           0.058262
Shift X                           0.057512
Abs error X                       0.044418
Contraction expansion ratio Y     0.037889
Contraction expansion ratio X     0.032633
Contraction expansion ratio XY    0.031480
Shift XY                          0.031477
dtype: float64
[12/12] features ████████████████████ Time elapsed: 0 secondsVariability Y                     0.215014
Variability XY                    0.158688
Variability X                     0.131570
Abs error Y                       0.094059
Abs error XY                      0.076149
Contraction expansion ratio X     0.065936
Shift Y                           0.058873
Shift X                           0.053880
Con

In [11]:
from sklearn.svm import SVC

model_svm = SVC(random_state=42)

results_svm = cross_validate(estimator=model_svm,
                          X=X,#X=features,
                          y=y,#y=labels,
                          cv=kfold,
                          scoring=scoring)
print(results_svm)

{'fit_time': array([0.02173018, 0.01208711, 0.01101995, 0.0108521 , 0.01003003,
       0.01042604, 0.01181269, 0.011585  , 0.01109385, 0.01052094]), 'score_time': array([0.00398684, 0.00288892, 0.00298595, 0.00283074, 0.00336599,
       0.00280809, 0.00348115, 0.00297809, 0.00282812, 0.00293994]), 'test_accuracy': array([0.82222222, 0.84444444, 0.86666667, 0.78888889, 0.76404494,
       0.83146067, 0.83146067, 0.86516854, 0.75280899, 0.83146067]), 'test_precision': array([0.86666667, 0.85365854, 0.87179487, 0.825     , 0.83333333,
       0.94444444, 0.84848485, 0.85714286, 0.80645161, 0.88888889]), 'test_recall': array([0.79591837, 0.81395349, 0.82926829, 0.73333333, 0.66666667,
       0.72340426, 0.73684211, 0.81081081, 0.6097561 , 0.74418605]), 'test_f1_score': array([0.82978723, 0.83333333, 0.85      , 0.77647059, 0.74074074,
       0.81927711, 0.78873239, 0.83333333, 0.69444444, 0.81012658])}


In [13]:
# https://chrisalbon.com/deep_learning/keras/k-fold_cross-validating_neural_networks/
from keras import models, layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def create_network():
    # network defination
    network = models.Sequential()
    # Add fully connected layer with a ReLU
    network.add(layers.Dense(units=16, activation='relu', input_shape=(12,)))
    network.add(layers.Dense(units=16, activation='relu'))
    network.add(layers.Dense(units=1, activation='sigmoid'))

    # compile network
    network.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    return network

Using TensorFlow backend.


In [14]:
# wrap keras model
neural_network = KerasClassifier(build_fn=create_network,
                                epochs=100,
                                batch_size=10,
                                verbose=1)

In [15]:
results_dnn = cross_validate(estimator=neural_network,
                          X=X,#X=features,
                          y=y,#y=labels,
                          cv=kfold,
                          scoring=scoring)
print(results_dnn)

Epoch 1/100
81/81 [==============================] - 0s 932us/step - loss: 0.6264 - accuracy: 0.6741
Epoch 2/100
81/81 [==============================] - 0s 958us/step - loss: 0.4694 - accuracy: 0.7848
Epoch 3/100
81/81 [==============================] - 0s 882us/step - loss: 0.4215 - accuracy: 0.7960
Epoch 4/100
81/81 [==============================] - 0s 950us/step - loss: 0.4049 - accuracy: 0.8060
Epoch 5/100
81/81 [==============================] - 0s 874us/step - loss: 0.3948 - accuracy: 0.8122
Epoch 6/100
81/81 [==============================] - 0s 967us/step - loss: 0.3883 - accuracy: 0.8172
Epoch 7/100
81/81 [==============================] - 0s 930us/step - loss: 0.3830 - accuracy: 0.8197
Epoch 8/100
81/81 [==============================] - 0s 940us/step - loss: 0.3809 - accuracy: 0.8246
Epoch 9/100
81/81 [==============================] - 0s 931us/step - loss: 0.3783 - accuracy: 0.8221
Epoch 10/100
81/81 [==============================] - 0s 931us/step - loss: 0.3763 - accura

81/81 [==============================] - 0s 873us/step - loss: 0.3433 - accuracy: 0.8507
Epoch 56/100
81/81 [==============================] - 0s 874us/step - loss: 0.3434 - accuracy: 0.8470
Epoch 57/100
81/81 [==============================] - 0s 884us/step - loss: 0.3427 - accuracy: 0.8483
Epoch 58/100
81/81 [==============================] - 0s 866us/step - loss: 0.3415 - accuracy: 0.8520
Epoch 59/100
81/81 [==============================] - 0s 929us/step - loss: 0.3408 - accuracy: 0.8495
Epoch 60/100
81/81 [==============================] - 0s 879us/step - loss: 0.3399 - accuracy: 0.8495
Epoch 61/100
81/81 [==============================] - 0s 872us/step - loss: 0.3404 - accuracy: 0.8470
Epoch 62/100
81/81 [==============================] - 0s 848us/step - loss: 0.3402 - accuracy: 0.8470
Epoch 63/100
81/81 [==============================] - 0s 857us/step - loss: 0.3396 - accuracy: 0.8495
Epoch 64/100
81/81 [==============================] - 0s 834us/step - loss: 0.3379 - accuracy: 

81/81 [==============================] - 0s 855us/step - loss: 0.3619 - accuracy: 0.8321
Epoch 15/100
81/81 [==============================] - 0s 852us/step - loss: 0.3596 - accuracy: 0.8396
Epoch 16/100
81/81 [==============================] - 0s 862us/step - loss: 0.3587 - accuracy: 0.8371
Epoch 17/100
81/81 [==============================] - 0s 832us/step - loss: 0.3580 - accuracy: 0.8433
Epoch 18/100
81/81 [==============================] - 0s 850us/step - loss: 0.3563 - accuracy: 0.8396
Epoch 19/100
81/81 [==============================] - 0s 889us/step - loss: 0.3559 - accuracy: 0.8396
Epoch 20/100
81/81 [==============================] - 0s 864us/step - loss: 0.3546 - accuracy: 0.8396
Epoch 21/100
81/81 [==============================] - 0s 863us/step - loss: 0.3525 - accuracy: 0.8420
Epoch 22/100
81/81 [==============================] - 0s 854us/step - loss: 0.3528 - accuracy: 0.8433
Epoch 23/100
81/81 [==============================] - 0s 843us/step - loss: 0.3515 - accuracy: 

81/81 [==============================] - 0s 886us/step - loss: 0.3222 - accuracy: 0.8634
Epoch 75/100
81/81 [==============================] - 0s 908us/step - loss: 0.3201 - accuracy: 0.8658
Epoch 76/100
81/81 [==============================] - 0s 876us/step - loss: 0.3211 - accuracy: 0.8658
Epoch 77/100
81/81 [==============================] - 0s 933us/step - loss: 0.3198 - accuracy: 0.8683
Epoch 78/100
81/81 [==============================] - 0s 860us/step - loss: 0.3201 - accuracy: 0.8671
Epoch 79/100
81/81 [==============================] - 0s 869us/step - loss: 0.3184 - accuracy: 0.8646
Epoch 80/100
81/81 [==============================] - 0s 879us/step - loss: 0.3196 - accuracy: 0.8683
Epoch 81/100
81/81 [==============================] - 0s 942us/step - loss: 0.3193 - accuracy: 0.8683
Epoch 82/100
81/81 [==============================] - 0s 856us/step - loss: 0.3182 - accuracy: 0.8683
Epoch 83/100
81/81 [==============================] - 0s 847us/step - loss: 0.3187 - accuracy: 

81/81 [==============================] - 0s 860us/step - loss: 0.3334 - accuracy: 0.8522
Epoch 55/100
81/81 [==============================] - 0s 868us/step - loss: 0.3317 - accuracy: 0.8571
Epoch 56/100
81/81 [==============================] - 0s 927us/step - loss: 0.3312 - accuracy: 0.8534
Epoch 57/100
81/81 [==============================] - 0s 866us/step - loss: 0.3305 - accuracy: 0.8609
Epoch 58/100
81/81 [==============================] - 0s 852us/step - loss: 0.3293 - accuracy: 0.8484
Epoch 59/100
81/81 [==============================] - 0s 847us/step - loss: 0.3302 - accuracy: 0.8584
Epoch 60/100
81/81 [==============================] - 0s 865us/step - loss: 0.3300 - accuracy: 0.8584
Epoch 61/100
81/81 [==============================] - 0s 839us/step - loss: 0.3293 - accuracy: 0.8596
Epoch 62/100
81/81 [==============================] - 0s 870us/step - loss: 0.3280 - accuracy: 0.8559
Epoch 63/100
81/81 [==============================] - 0s 845us/step - loss: 0.3281 - accuracy: 

81/81 [==============================] - 0s 851us/step - loss: 0.3455 - accuracy: 0.8410
Epoch 35/100
81/81 [==============================] - 0s 862us/step - loss: 0.3438 - accuracy: 0.8422
Epoch 36/100
81/81 [==============================] - 0s 837us/step - loss: 0.3435 - accuracy: 0.8410
Epoch 37/100
81/81 [==============================] - 0s 859us/step - loss: 0.3424 - accuracy: 0.8422
Epoch 38/100
81/81 [==============================] - 0s 863us/step - loss: 0.3432 - accuracy: 0.8422
Epoch 39/100
81/81 [==============================] - 0s 839us/step - loss: 0.3422 - accuracy: 0.8447
Epoch 40/100
81/81 [==============================] - 0s 853us/step - loss: 0.3416 - accuracy: 0.8447
Epoch 41/100
81/81 [==============================] - 0s 877us/step - loss: 0.3405 - accuracy: 0.8422
Epoch 42/100
81/81 [==============================] - 0s 866us/step - loss: 0.3415 - accuracy: 0.8435
Epoch 43/100
81/81 [==============================] - 0s 837us/step - loss: 0.3401 - accuracy: 

81/81 [==============================] - 0s 857us/step - loss: 0.3729 - accuracy: 0.8186
Epoch 15/100
81/81 [==============================] - 0s 848us/step - loss: 0.3704 - accuracy: 0.8174
Epoch 16/100
81/81 [==============================] - 0s 910us/step - loss: 0.3684 - accuracy: 0.8211
Epoch 17/100
81/81 [==============================] - 0s 916us/step - loss: 0.3681 - accuracy: 0.8174
Epoch 18/100
81/81 [==============================] - 0s 1ms/step - loss: 0.3665 - accuracy: 0.8236
Epoch 19/100
81/81 [==============================] - 0s 884us/step - loss: 0.3662 - accuracy: 0.8199
Epoch 20/100
81/81 [==============================] - 0s 867us/step - loss: 0.3647 - accuracy: 0.8224
Epoch 21/100
81/81 [==============================] - 0s 847us/step - loss: 0.3639 - accuracy: 0.8273
Epoch 22/100
81/81 [==============================] - 0s 857us/step - loss: 0.3630 - accuracy: 0.8298
Epoch 23/100
81/81 [==============================] - 0s 868us/step - loss: 0.3615 - accuracy: 0.

81/81 [==============================] - 0s 948us/step - loss: 0.3158 - accuracy: 0.8596
Epoch 75/100
81/81 [==============================] - 0s 872us/step - loss: 0.3150 - accuracy: 0.8621
Epoch 76/100
81/81 [==============================] - 0s 880us/step - loss: 0.3125 - accuracy: 0.8634
Epoch 77/100
81/81 [==============================] - 0s 851us/step - loss: 0.3134 - accuracy: 0.8621
Epoch 78/100
81/81 [==============================] - 0s 888us/step - loss: 0.3140 - accuracy: 0.8571
Epoch 79/100
81/81 [==============================] - 0s 858us/step - loss: 0.3131 - accuracy: 0.8609
Epoch 80/100
81/81 [==============================] - 0s 868us/step - loss: 0.3112 - accuracy: 0.8634
Epoch 81/100
81/81 [==============================] - 0s 874us/step - loss: 0.3095 - accuracy: 0.8658
Epoch 82/100
81/81 [==============================] - 0s 845us/step - loss: 0.3123 - accuracy: 0.8621
Epoch 83/100
81/81 [==============================] - 0s 846us/step - loss: 0.3112 - accuracy: 

81/81 [==============================] - 0s 907us/step - loss: 0.3397 - accuracy: 0.8484
Epoch 55/100
81/81 [==============================] - 0s 1ms/step - loss: 0.3391 - accuracy: 0.8472
Epoch 56/100
81/81 [==============================] - 0s 871us/step - loss: 0.3400 - accuracy: 0.8484
Epoch 57/100
81/81 [==============================] - 0s 853us/step - loss: 0.3379 - accuracy: 0.8472
Epoch 58/100
81/81 [==============================] - 0s 852us/step - loss: 0.3386 - accuracy: 0.8472
Epoch 59/100
81/81 [==============================] - 0s 864us/step - loss: 0.3365 - accuracy: 0.8497
Epoch 60/100
81/81 [==============================] - 0s 901us/step - loss: 0.3375 - accuracy: 0.8497
Epoch 61/100
81/81 [==============================] - 0s 866us/step - loss: 0.3353 - accuracy: 0.8522
Epoch 62/100
81/81 [==============================] - 0s 924us/step - loss: 0.3349 - accuracy: 0.8534
Epoch 63/100
81/81 [==============================] - 0s 942us/step - loss: 0.3354 - accuracy: 0.

In [16]:
print('Deep Neural Network Metrics:')
print('Fit time:',results_dnn['fit_time'])
print('Score time:',results_dnn['score_time'])
print('10-fold Accuracy:',results_dnn['test_accuracy'])
print('Accuracy(Mean (Standard Deviation)): %f (%f)'%(np.mean(results_dnn['test_accuracy']),np.std(results_dnn['test_accuracy'])))
print('10-fold Precision:',results_dnn['test_precision'])
print('Precision(Mean (Standard Deviation): %f (%f)'%(np.mean(results_dnn['test_precision']),np.std(results_dnn['test_precision'])))
print('10-fold Recall:',results_dnn['test_recall'])
print('Recall(Mean (Standard Deviation): %f (%f)'%(np.mean(results_dnn['test_recall']),np.std(results_dnn['test_recall'])))
print('10-fold f1-score:',results_dnn['test_f1_score'])
print('f1-score(Mean (Standard Deviation): %f (%f)'%(np.mean(results_dnn['test_f1_score']),np.std(results_dnn['test_f1_score'])))

Deep Neural Network Metrics:
Fit time: [8.39762998 7.86780524 7.75316095 7.72562814 7.76973701 7.8308208
 7.74413204 7.94652104 7.78784895 7.78154969]
Score time: [0.11622405 0.05142784 0.0472641  0.05026603 0.05128694 0.05082703
 0.05285192 0.05252695 0.05386209 0.04972029]
10-fold Accuracy: [0.82222222 0.86666667 0.82222222 0.82222222 0.83146067 0.83146067
 0.84269663 0.85393258 0.73033708 0.82022472]
Accuracy(Mean (Standard Deviation)): 0.824345 (0.034601)
10-fold Precision: [0.85106383 0.8974359  0.80487805 0.87179487 0.84090909 0.97058824
 0.9        0.83333333 0.71794872 0.81395349]
Precision(Mean (Standard Deviation): 0.850191 (0.063954)
10-fold Recall: [0.81632653 0.81395349 0.80487805 0.75555556 0.82222222 0.70212766
 0.71052632 0.81081081 0.68292683 0.81395349]
Recall(Mean (Standard Deviation): 0.773328 (0.052386)
10-fold f1-score: [0.83333333 0.85365854 0.80487805 0.80952381 0.83146067 0.81481481
 0.79411765 0.82191781 0.7        0.81395349]
f1-score(Mean (Standard Deviation

In [17]:
print('Logistic Regression: Accuracy(Mean (Standard Deviation)): %f (%f)'%(np.mean(results_lr['test_accuracy']),np.std(results_lr['test_accuracy'])))
print('Decision Tree: Accuracy(Mean (Standard Deviation)): %f (%f)'%(np.mean(results_dt['test_accuracy']),np.std(results_dt['test_accuracy'])))
print('Random Forest: Accuracy(Mean (Standard Deviation)): %f (%f)'%(np.mean(results_rf['test_accuracy']),np.std(results_rf['test_accuracy'])))
print('Random Forest with Hyperparameters Tuning: Accuracy(Mean (Standard Deviation)): %f (%f)'%(np.mean(results_rft['test_accuracy']),np.std(results_rft['test_accuracy'])))
print('Support Vector Machine: Accuracy(Mean (Standard Deviation)): %f (%f)'%(np.mean(results_svm['test_accuracy']),np.std(results_svm['test_accuracy'])))
print('Deep Neural Network: Accuracy(Mean (Standard Deviation)): %f (%f)'%(np.mean(results_dnn['test_accuracy']),np.std(results_dnn['test_accuracy'])))

Logistic Regression: Accuracy(Mean (Standard Deviation)): 0.821011 (0.041444)
Decision Tree: Accuracy(Mean (Standard Deviation)): 0.822122 (0.034194)
Random Forest: Accuracy(Mean (Standard Deviation)): 0.829950 (0.033242)
Random Forest with Hyperparameters Tuning: Accuracy(Mean (Standard Deviation)): 0.827690 (0.032703)
Support Vector Machine: Accuracy(Mean (Standard Deviation)): 0.819863 (0.037186)
Deep Neural Network: Accuracy(Mean (Standard Deviation)): 0.824345 (0.034601)
